In [1]:
import sqlite3
import pandas as pd
import random
import numpy as np

# Step 1: Simulate query data
def simulate_query_data(num_queries=1000):
    queries = []
    for i in range(num_queries):
        table_size = random.randint(1000, 100000)  # in rows
        num_joins = random.randint(1, 5)
        index_used = random.choice([0, 1])
        query_complexity = random.uniform(0.1, 1.0) * num_joins
        execution_time = query_complexity * table_size * (1 - 0.3 * index_used)
        queries.append([table_size, num_joins, index_used, query_complexity, execution_time])

    df = pd.DataFrame(queries, columns=['table_size', 'num_joins', 'index_used', 'query_complexity', 'execution_time'])
    return df

query_data = simulate_query_data()
print(query_data.head())


   table_size  num_joins  index_used  query_complexity  execution_time
0       10045          4           0          3.207851    32222.863980
1       56487          5           1          1.935499    76531.361886
2       52794          4           0          3.966140   209388.377075
3       71816          3           0          1.949840   140029.730797
4       48515          4           0          1.827594    88665.744433


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Step 3: Split the data into training and testing sets
X = query_data[['table_size', 'num_joins', 'index_used', 'query_complexity']]
y = query_data['execution_time']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 5: Test the model and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 58304824.48618544


In [4]:
def optimize_query(table_size, num_joins, index_used):
    query_complexity = num_joins * random.uniform(0.1, 1.0)

    # Step 6: Create a DataFrame with the same feature names as the training data
    features = pd.DataFrame([[table_size, num_joins, index_used, query_complexity]],
                            columns=['table_size', 'num_joins', 'index_used', 'query_complexity'])

    # Step 7: Predict execution time using the trained model
    predicted_time = model.predict(features)[0]

    # Step 8: Suggest an optimized query plan (e.g., using index if not used)
    if index_used == 0:
        alternative_time_with_index = predicted_time * 0.7  # Assume index reduces time by 30%
        if alternative_time_with_index < predicted_time:
            return f"Use Index: Predicted Time = {alternative_time_with_index:.2f} ms"

    return f"Predicted Execution Time: {predicted_time:.2f} ms"

# Example usage
print(optimize_query(table_size=5000, num_joins=3, index_used=0))
print(optimize_query(table_size=30000, num_joins=2, index_used=1))


Use Index: Predicted Time = 8976.18 ms
Predicted Execution Time: 13535.36 ms
